# Learn 101


This tutorial is supposed to help you to familiarize with paasify. The idea is to simply deploy 2 apps: paperless-ng and homepage. For this purpose, we will deploy a proxy, Traefik, in front of it in way to serve those 2 apps.

Let's start!

## Create new project

In [1]:
# Please ignore this section
ROOT_DIR=${ROOT_DIR:-$PWD}
cd $ROOT_DIR
rm -rf examples/ex1 || true
export NOTTY=true
export LANG=C

Let's create a fresh dir where we can create our project:

In [2]:
mkdir -p examples/ex1
cd examples/ex1

## Create project

Then, we want to create a new paasify project, let's call it `devbox`

In [3]:
paasify new devbox

  NOTICE: Creating: 'devbox/paasify.yml' from '/home/jez/volumes/data/prj/mrjk/POC/paasify_transiant/paasify/assets/paasify.yml'
  NOTICE: Creating: 'devbox/.gitignore' from '/home/jez/volumes/data/prj/mrjk/POC/paasify_transiant/paasify/assets/gitignore'
  NOTICE: New project updated in: devbox


Yipeeh, our first project has been created, and 2 files has been created: `paasify.yml` and `.gitignore`. Let's inspect them.

In [4]:
cd devbox
tail -n 999 paasify.yml .gitignore


==> paasify.yml <==
config:

  # Load extra vars
  # -----------------
  # extra_vars:
  #   - ../vars.extra.yml

  vars:

    # Determine your application domain
    # app_domain: devbox.home

    # Default network name for all stacks
    # app_network_name: $net_proxy_public

sources:

  # Add you sources below
  # - name: default
  #   remote: git@github.com:user/repo.git
  #   #remote: ../../my_local_collection

stacks:

  # Add you stacks below
  # - app: httpd
  #   vars:
  #     app_name: web

==> .gitignore <==
.paasify/collections/*
*/data/*
*/share/*
*/tmp/*
*/db_data/*


Ok, it seems the `paasify.yml` file want to be edited, but for this time, we will just replace it by another one:

In [5]:
cat << 'EOF' > paasify.yml
# My new project

sources:
  - name: community
    remote: https://github.com/barbu-it/paasify-collection-community

config:
  vars:
    app_expose_ip: 127.0.0.1
    app_domain: ${app_expose_ip}.nip.io
    traefik_net_name: devbox_proxy

  tags_prefix:
    - homepage
    - traefik-svc

stacks:
  - app: proxy
    vars:
      app_network_name: $traefik_net_name
    tags:
      - expose_http
  - app: home
  - app: community:paperless-ng

EOF

# cat paasify.yml


## Install project sources

Ok, our project is ready, let's fetch its source, as you seen in the config, we refers to a git URL. We need to download/install this source locally before being enable to use it. There is one command for this purpose:

In [6]:
paasify src install

  NOTICE: Source 'paasify' is a path, no need to install
  NOTICE: Installing 'community' git: https://github.com/barbu-it/paasify-collection-community in .paasify/collections/barbu-it-paasify-collection-community
Cloning into '.paasify/collections/barbu-it-paasify-collection-community'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 42 (delta 0), reused 42 (delta 0), pack-reused 0
Receiving objects: 100% (42/42), 66.68 KiB | 1.39 MiB/s, done.


Now our source has been installed, let's check it:

In [7]:
paasify src ls

  NAME                 INSTALLED  GIT      REMOTE                                             PATH
  paasify              True       False                                                       /home/jez/volumes/data/prj/mrjk/POC/paasify_transiant/paasify/assets/collections/paasify
  community            True       True     https://github.com/barbu-it/paasify-collection-community .paasify/collections/barbu-it-paasify-collection-community


## Start stacks

Well, we have our config, our source installed, let's deploy, for this, one command:

In [8]:
paasify apply

  NOTICE: Apply stacks
  NOTICE: Assemble stack: proxy
  NOTICE: Assemble stack: home
  NOTICE: Assemble stack: paperless-ng
  NOTICE:   Start stack: proxy
  NOTICE:   Start stack: home
  NOTICE:   Start stack: paperless-ng
  NOTICE: Stack has been applied


Then wait a bit that docker download the apps image and then you can check your applications:
* http://home.127.0.0.1.nip.io/
* http://proxy.127.0.0.1.nip.io/dashboard/
* http://paperless.127.0.0.1.nip.io/ (password aplication is admin/admin)

Yeah! You can toy a bit with those apps, and let's continue our walkthrough. Actually, what is running ? You have the `ps` command for that:

In [9]:
paasify ps

  devbox_proxy                     0b6239eaa413 devbox_proxy-traefik-1                   traefik          running    127.0.0.1:80->80/tcp
  devbox_home                      30c398d74a43 devbox_home-homepage-1                   homepage         running    
  devbox_paperless-ng              cd32a7782180 devbox_paperless-ng-paperless-1          paperless        running    


So basically, we see that we have 3 containers running, and the proxy is listening on 127.0.0.1 on port 80, and it is here to reroute all traffic trough paperless and homepage instances. This is exactly what we want, all our web services behind a proxy. Later, it will be possible to use your own domain, set login/password proection, add let's encrypt HTTPS ...

But for now, let's remove everything like you never did anything:

In [10]:
paasify down

  NOTICE:   Stop stack: paperless-ng
  NOTICE:   Stop stack: home
  NOTICE:   Stop stack: proxy


As you can see, there is no more output

In [11]:
paasify ps

So, even the containers has been destroyed, there are no tracks of them anymore. However, you still have your data:

In [12]:
tree

.
|-- home
|   |-- conf
|   `-- docker-compose.run.yml
|-- paasify.yml
|-- paperless-ng
|   |-- conf
|   |   |-- custom-cont-init.d
|   |   |-- custom-services.d
|   |   |-- db.sqlite3
|   |   `-- log
|   |-- data
|   |   |-- consume
|   |   `-- media
|   |       `-- documents
|   |           |-- originals
|   |           `-- thumbnails
|   `-- docker-compose.run.yml
`-- proxy
    |-- conf
    |-- data
    |-- docker-compose.run.yml
    `-- logs

17 directories, 5 files


To put back online the apps, just run again `paasify up`. If you want to modify your `paasify.yml`, then don't forget to `apply` on your stacks (Note: due to a docker "bug", labels changes may need an actual `recreate` instead of `apply`).

Before leaving, we will remove the project. If you made some modifications into the app, they will be lost after this command.

In [13]:
cd $ROOT_DIR
rm -rf examples/ex1

And this all for this first tutorial :)